# Expressions

Rational expressions, or _expressions_ for short, denote (rational) languages in a compact way.  Since Vcsn supports weighted expressions, they actually can denoted rational series.

This page document the syntax and transformations (called _identities_) that are applied to every expression.  The list of available algorithms using expression is in the [Algorithms page](Algorithms.ipynb).

## Syntax
The syntax for rational expressions is as follows (with increasing precedence):
- `\z`, the empty expression
- `\e`, the empty word
- `a`, the word `a`
- `e+f`, the addition (disjunction, union) of `e` and `f` (note the use of `+`, `|` is not accepted)
- `e&f`, the conjunction (intersection) of `e` and `f`
- `ef` and `e.f`, the multiplication (concatenation) of `e` and `f`
- `<k>e`, the left exterior product (left-scalar product) of `e` by `k`
- `e<k>`, the right exterior product (right-scalar product) of `e` by `k`
- `e*` and `e{*}`, any number of repetitions of `e` (the Kleene closure of `e`)
- `e{n}`, the power (repeated multiplication) of `e` `n` times: `ee ... e`
- `e{n,m}`, any repetition of `e` between `n` and `m`, i.e., the sum of the powers of `e` between `n` and `m`: `e{n}+e{n+1}+ ... +e{m}`
- `e{n,}`, the sum of powers of `e` at least `n` times: `e{n}e*`
- `e{,m}`, at most `m` repetitions of `e`: `e{0,m}`
- `e{+}`, at least one `e`: `e{1,}`
- `e{?}`, `e` optional: `e{0,1}`
- `e{c}`, the complement of `e`

where `e` and `f` denote expressions, `a` a label, `k` a weight, and `n` and `m` natural numbers.

## Identities
Some rewriting rules are applied on the expressions to "simplify" them.  The strength of this simplification is graduated.

### Trivial Identities

### Traditional Identities

### Series Identities

## Examples


In [1]:
import vcsn
from IPython.display import Latex
import numpy as np
import pandas as pd
pd.options.display.max_colwidth=0

:0: FutureWarning: IPython widgets are experimental and may change in the future.


<IPython.core.display.Javascript object>

In [2]:
ids = ['trivial', 'traditional', 'series']
def example(*es):
    c = vcsn.context('lal_char, b')
    res = []
    for e in es:
        res.append([e] + ['$' + c.expression(e, id).format('latex') + '$' for id in ids])
    return pd.DataFrame(res, columns=['Input', 'Trivial', 'Traditional', 'Series'])

In [3]:
example('a', 'b+a', '[ab][ab]')

,Input,Trivial,Traditional,Series
0,a,$a$,$a$,$a$
1,b+a,$b + a$,$a + b$,$a \oplus b$
2,[ab][ab],"$\left(a + b\right) \, \left(a + b\right)$","$\left(a + b\right) \, \left(a + b\right)$","$a \, a \oplus a \, b \oplus b \, a \oplus b \, b$"
